In [3]:
import os
import json
import yfinance as yf
import feedparser
import gradio as gr

from datetime import datetime, timedelta
import pytz
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# =====================================================
# ENV SETUP
# =====================================================
load_dotenv()

llm = ChatOpenAI(
    model="openai/gpt-oss-120b",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# =====================================================
# INTENT EXTRACTION
# =====================================================
intent_prompt = PromptTemplate.from_template("""
Extract intent, Indian company name, and BSE ticker.

Rules:
- Use BSE ticker ending with .BO
- Example: Reliance Industries → RELIANCE.BO

Query: {query}

Return STRICT JSON:
{{
  "intent": "why | when | trend",
  "company": "",
  "ticker": ""
}}
""")

intent_chain = intent_prompt | llm

def extract_intent(query: str):
    response = intent_chain.invoke({"query": query})
    return json.loads(response.content)

# =====================================================
# MARKET ANALYSIS
# =====================================================
def market_analysis(ticker):
    if not ticker:
        return "No valid Indian stock ticker detected."

    ist = pytz.timezone("Asia/Kolkata")
    intraday = yf.download(
        ticker,
        period="2d",
        interval="5m",
        progress=False
    )

    if not intraday.empty:
        last_time = intraday.index[-1].tz_convert(ist)
        price = float(intraday["Close"].iloc[-1])
        return (
            f"Latest price: ₹{round(price,2)} "
            f"(Updated {last_time.strftime('%d %b %Y %H:%M IST')})"
        )

    daily = yf.download(ticker, period="7d", progress=False)
    if daily.empty:
        return "No recent market data available."

    close_price = float(daily["Close"].iloc[-1])
    last_day = daily.index[-1].date()

    return f"Market closed. Last close on {last_day}: ₹{round(close_price,2)}"

# =====================================================
# NEWS + VECTOR SEARCH
# =====================================================
NEWS_SOURCES = [
    "https://economictimes.indiatimes.com/markets/rssfeeds/1977021501.cms",
    "https://www.moneycontrol.com/rss/markets.xml"
]

def fetch_news(rss_urls, limit=5):
    articles = []
    for url in rss_urls:
        feed = feedparser.parse(url)
        for entry in feed.entries[:limit]:
            articles.append(entry.title + ". " + entry.get("summary", ""))
    return articles

def build_vector_store(texts):
    return FAISS.from_texts(texts, embeddings)

def retrieve_context(query, vectorstore, k=3):
    docs = vectorstore.similarity_search(query, k=k)
    return "\n".join(d.page_content for d in docs)

# =====================================================
# FORMATTED EXPLANATION PROMPT
# =====================================================
explain_prompt = PromptTemplate.from_template("""
You are an Indian stock market analyst.

Respond ONLY in clean, professional MARKDOWN.

Follow EXACTLY this structure:

## 📌 Question
{query}

## 📊 Market Snapshot
- {market_info}

## 📰 Key News & Events
- Concise bullet points
- Only relevant information

## 📈 Analysis & Explanation
- Simple language
- Short paragraphs
- Indian market context

## 🧠 Investor Takeaway
- 2–3 clear bullet points

News Context:
{context}
""")

explain_chain = explain_prompt | llm

def explain(query, market_info, context):
    response = explain_chain.invoke({
        "query": query,
        "market_info": market_info,
        "context": context
    })
    return response.content.strip()

# =====================================================
# MAIN CHATBOT FUNCTION (WITH STATUS)
# =====================================================
def analytical_chatbot(query, progress=gr.Progress()):
    progress(0, desc="🔍 Understanding your question")
    parsed = extract_intent(query)

    progress(0.3, desc="📊 Fetching market data")
    ticker = parsed.get("ticker", "")
    market_info = market_analysis(ticker)

    progress(0.6, desc="📰 Analyzing latest news")
    news_texts = fetch_news(NEWS_SOURCES)
    vectorstore = build_vector_store(news_texts)
    context = retrieve_context(query, vectorstore)

    progress(0.9, desc="🧠 Generating insights")
    answer = explain(query, market_info, context)

    progress(1.0, desc="✅ Done")
    return answer

# =====================================================
# GRADIO UI (LOADING SPINNER ENABLED)
# =====================================================
with gr.Blocks(title="Indian Stock Market AI") as demo:
    gr.Markdown("""
# 🇮🇳 Indian Stock Market Analytical Assistant

Ask questions like:
- **Why is Reliance stock falling today?**
- **Trend of TCS share price**
- **When will Infosys recover?**
""")

    query = gr.Textbox(
        label="Your Question",
        placeholder="Ask a stock market question...",
        lines=2
    )

    output = gr.Markdown(label="📊 AI Market Analysis")

    btn = gr.Button("Analyze", variant="primary")

    btn.click(
        fn=analytical_chatbot,
        inputs=query,
        outputs=output,
        show_progress=True
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


C:\Users\Rohan\AppData\Local\Temp\ipykernel_17548\694232628.py:75: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(intraday["Close"].iloc[-1])
